# My Capstone Project 02
## Segmenting and Clustering Neighborhoods in Toronto
### Author Eloisa M. de Lira¶

In [1]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from bs4 import BeautifulSoup

import geocoder # import geocoder

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [3]:
url= 'https://cocl.us/Geospatial_data'
postal_code=pd.read_csv(url)



In [4]:
!wget -q -O 'canada_data.json' https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=947772202
print('Data downloaded!')
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
neighborhood_row=pd.DataFrame(df[0])
canada_neighborhood = neighborhood_row[neighborhood_row['Borough'] != 'Not assigned'].reset_index(drop=True)
canada_neighborhood['Neighborhood']= canada_neighborhood.Neighborhood.str.replace('Regent Park / Harbourfront','Regent Park , Harbourfront')
canada_neighborhood.replace(canada_neighborhood['Neighborhood']== 'NaN', canada_neighborhood['Borough'])
canada_merged = canada_neighborhood.merge(postal_code, left_on='Postal code', right_on='Postal Code',how='inner')
canada_merged.drop('Postal Code',axis=1)


Data downloaded!


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [5]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Canada are 43.6534817, -79.3839347.


In [6]:
# create map of Canada using latitude and longitude values
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(canada_merged['Latitude'],canada_merged['Longitude'], canada_merged['Borough'], canada_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False).add_to(map_canada)  
    
map_canada


``Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.``

``Just make sure:``

``(1)to add enough Markdown cells to explain what you decided to do and to report any observations you make.``
``(2)to generate maps to visualize your neighborhoods and how they cluster together.``

In [7]:
#define credentials for FOURSQUARE

CLIENT_ID = 'TLFIL03HBNKTPUOIVUIGE1M010WJNZZOHFU1KMZ2DVBZR2N5' # your Foursquare ID
CLIENT_SECRET = 'PEDBVRQQ024W1K1E50PT5W4BTRQWWIMN3QJXTPBS4Z0RCGUO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TLFIL03HBNKTPUOIVUIGE1M010WJNZZOHFU1KMZ2DVBZR2N5
CLIENT_SECRET:PEDBVRQQ024W1K1E50PT5W4BTRQWWIMN3QJXTPBS4Z0RCGUO


In [8]:
#Explore de neighborhood from Toronto

LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:


toronto_venues = getNearbyVenues(names=canada_merged['Neighborhood'],
                                   latitudes=canada_merged['Latitude'],
                                   longitudes=canada_merged['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [10]:
print(toronto_venues.shape)
Toronto_venues=toronto_venues.rename(columns={'Neighborhood Latitude': 'Latitude', 'Neighborhood Longitude': 'Longitude'})
Toronto_venues.head()

(2197, 7)


,Neighborhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [11]:
Toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


In [12]:
#Analize each neighborhood

# one hot encoding method
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
toronto_onehot.shape
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()





In [13]:
#neighborhood along with the top 10 most common venues

num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                        venue  freq
0                      Lounge  0.25
1                Skating Rink  0.25
2   Latin American Restaurant  0.25
3              Breakfast Spot  0.25
4                 Men's Store  0.00
5  Modern European Restaurant  0.00
6           Mobile Phone Shop  0.00
7          Miscellaneous Shop  0.00
8   Middle Eastern Restaurant  0.00
9          Mexican Restaurant  0.00


----Alderwood / Long Branch----
                venue  freq
0         Pizza Place  0.22
1            Pharmacy  0.11
2      Sandwich Place  0.11
3        Skating Rink  0.11
4                 Gym  0.11
5                 Pub  0.11
6                Pool  0.11
7         Coffee Shop  0.11
8   Mobile Phone Shop  0.00
9  Miscellaneous Shop  0.00


----Bathurst Manor / Wilson Heights / Downsview North----
                       venue  freq
0                       Bank  0.11
1                Coffee Shop  0.11
2              Shopping Mall  0.05
3                 Restaurant  0.05
4        

                        venue  freq
0           Indian Restaurant  0.33
1                   Pet Store  0.17
2       Vietnamese Restaurant  0.17
3                     Brewery  0.17
4          Chinese Restaurant  0.17
5                 Yoga Studio  0.00
6          Mexican Restaurant  0.00
7  Modern European Restaurant  0.00
8           Mobile Phone Shop  0.00
9          Miscellaneous Shop  0.00


----Downsview----
                  venue  freq
0         Grocery Store  0.19
1                  Park  0.12
2          Home Service  0.06
3    Athletics & Sports  0.06
4                  Bank  0.06
5                 Hotel  0.06
6          Liquor Store  0.06
7        Discount Store  0.06
8  Gym / Fitness Center  0.06
9         Shopping Mall  0.06


----Dufferin / Dovercourt Village----
                       venue  freq
0                     Bakery  0.11
1                   Pharmacy  0.11
2                       Pool  0.06
3                       Park  0.06
4                       Café  0.06
5  M

                           venue  freq
0                           Café  0.07
1          Vietnamese Restaurant  0.05
2                    Coffee Shop  0.05
3                            Bar  0.05
4            Dumpling Restaurant  0.04
5  Vegetarian / Vegan Restaurant  0.04
6             Mexican Restaurant  0.04
7                   Burger Joint  0.03
8                         Bakery  0.03
9                    Pizza Place  0.03


----Kingsview Village / St. Phillips / Martin Grove Gardens / Richview Gardens----
                       venue  freq
0                       Park  0.25
1          Mobile Phone Shop  0.25
2                Pizza Place  0.25
3             Sandwich Place  0.25
4   Mediterranean Restaurant  0.00
5         Miscellaneous Shop  0.00
6  Middle Eastern Restaurant  0.00
7         Mexican Restaurant  0.00
8                Men's Store  0.00
9             Medical Center  0.00


----Lawrence Manor / Lawrence Heights----
                             venue  freq
0               

                 venue  freq
0          Coffee Shop  0.24
1          Yoga Studio  0.03
2   Italian Restaurant  0.03
3         Burger Joint  0.03
4                 Café  0.03
5             Boutique  0.03
6  Fried Chicken Joint  0.03
7           Restaurant  0.03
8             Beer Bar  0.03
9          Music Venue  0.03


----Regent Park , Harbourfront----
                venue  freq
0         Coffee Shop  0.18
1              Bakery  0.07
2                 Pub  0.07
3                Park  0.07
4                Café  0.04
5      Breakfast Spot  0.04
6  Mexican Restaurant  0.04
7          Beer Store  0.02
8   French Restaurant  0.02
9                 Spa  0.02


----Richmond / Adelaide / King----
             venue  freq
0             Café  0.06
1      Coffee Shop  0.06
2       Restaurant  0.06
3              Bar  0.03
4  Thai Restaurant  0.03
5              Gym  0.03
6           Bakery  0.03
7   Breakfast Spot  0.02
8   Clothing Store  0.02
9        Bookstore  0.02


----Rosedale----
     

                 venue  freq
0          Coffee Shop  0.15
1                 Café  0.08
2                Hotel  0.07
3           Restaurant  0.06
4  American Restaurant  0.03
5                  Bar  0.03
6   Seafood Restaurant  0.03
7            Gastropub  0.03
8          Pizza Place  0.02
9             Beer Bar  0.02


----University of Toronto / Harbord----
                 venue  freq
0                 Café  0.17
1           Restaurant  0.06
2            Bookstore  0.06
3                  Bar  0.06
4               Bakery  0.06
5  Japanese Restaurant  0.06
6    French Restaurant  0.03
7           Beer Store  0.03
8             Beer Bar  0.03
9   Chinese Restaurant  0.03


----Victoria Village----
                             venue  freq
0                      Coffee Shop   0.2
1                      Pizza Place   0.2
2                     Hockey Arena   0.2
3                     Intersection   0.2
4            Portuguese Restaurant   0.2
5        Middle Eastern Restaurant   0.0
6     

In [14]:
#function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
#create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Women's Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,Alderwood / Long Branch,Pizza Place,Pub,Gym,Skating Rink,Coffee Shop,Pharmacy,Pool,Sandwich Place,Distribution Center,Dessert Shop
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Frozen Yogurt Shop,Shopping Mall,Sandwich Place,Bridal Shop,Restaurant,Diner,Deli / Bodega,Supermarket
3,Bayview Village,Japanese Restaurant,Chinese Restaurant,Bank,Café,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Diner
4,Bedford Park / Lawrence Manor East,Sandwich Place,Pizza Place,Coffee Shop,Restaurant,Italian Restaurant,Comfort Food Restaurant,Café,Indian Restaurant,Butcher,Sushi Restaurant


# Let's cluster!

In [16]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:72] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 6, 3, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 2,
       1, 4, 1, 1, 5, 1], dtype=int32)

``create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood. ``

In [17]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)
neighborhoods_venues_sorted.head()
#neighborhoods_venues_sorted['Cluster Labels'] = neighborhoods_venues_sorted['Cluster Labels']
toronto_merged = canada_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



In [24]:
toronto_merged.head(22)

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,2.0,Park,Fireworks Store,Food & Drink Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,1.0,Hockey Arena,Intersection,Pizza Place,Coffee Shop,Portuguese Restaurant,Women's Store,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636,1.0,Coffee Shop,Pub,Park,Bakery,Mexican Restaurant,Breakfast Spot,Café,Ice Cream Shop,Electronics Store,Hotel
3,M6A,North York,Lawrence Manor / Lawrence Heights,M6A,43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Accessories Store,Vietnamese Restaurant,Convenience Store,Concert Hall
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,M7A,43.662301,-79.389494,1.0,Coffee Shop,Yoga Studio,Café,Beer Bar,Italian Restaurant,Juice Bar,Boutique,Burger Joint,Burrito Place,Restaurant
5,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,Malvern / Rouge,M1B,43.806686,-79.194353,0.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Dim Sum Restaurant
7,M3B,North York,Don Mills,M3B,43.745906,-79.352188,1.0,Restaurant,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Gym / Fitness Center,Sporting Goods Shop,Bike Shop,Sandwich Place,Discount Store
8,M4B,East York,Parkview Hill / Woodbine Gardens,M4B,43.706397,-79.309937,1.0,Pizza Place,Gym / Fitness Center,Gastropub,Fast Food Restaurant,Bus Line,Intersection,Café,Bank,Athletics & Sports,Breakfast Spot
9,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,1.0,Clothing Store,Coffee Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Cosmetics Shop,Japanese Restaurant,Restaurant,Tea Room,Bookstore


``Note we have some Nan values...let's remove it``

In [34]:
toronto_merged=toronto_merged.dropna(subset=['ClusterLabels'])
toronto_merged.head(9)

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,2.0,Park,Fireworks Store,Food & Drink Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,1.0,Hockey Arena,Intersection,Pizza Place,Coffee Shop,Portuguese Restaurant,Women's Store,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636,1.0,Coffee Shop,Pub,Park,Bakery,Mexican Restaurant,Breakfast Spot,Café,Ice Cream Shop,Electronics Store,Hotel
3,M6A,North York,Lawrence Manor / Lawrence Heights,M6A,43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Accessories Store,Vietnamese Restaurant,Convenience Store,Concert Hall
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,M7A,43.662301,-79.389494,1.0,Coffee Shop,Yoga Studio,Café,Beer Bar,Italian Restaurant,Juice Bar,Boutique,Burger Joint,Burrito Place,Restaurant
6,M1B,Scarborough,Malvern / Rouge,M1B,43.806686,-79.194353,0.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Dim Sum Restaurant
7,M3B,North York,Don Mills,M3B,43.745906,-79.352188,1.0,Restaurant,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Gym / Fitness Center,Sporting Goods Shop,Bike Shop,Sandwich Place,Discount Store
8,M4B,East York,Parkview Hill / Woodbine Gardens,M4B,43.706397,-79.309937,1.0,Pizza Place,Gym / Fitness Center,Gastropub,Fast Food Restaurant,Bus Line,Intersection,Café,Bank,Athletics & Sports,Breakfast Spot
9,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,1.0,Clothing Store,Coffee Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Cosmetics Shop,Japanese Restaurant,Restaurant,Tea Room,Bookstore


In [37]:
#toronto_merged['ClusterLabels']=toronto_merged.loc['ClusterLabels'].astype(int)

# Mapping now these clusters

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster=int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

``Examiner clusters``

In [45]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,-79.194353,0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Dim Sum Restaurant
27,North York,-79.363452,0,Fast Food Restaurant,Golf Course,Dog Run,Mediterranean Restaurant,Pool,Women's Store,Donut Shop,Discount Store,Distribution Center,Doner Restaurant


In [46]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,-79.315572,1,Hockey Arena,Intersection,Pizza Place,Coffee Shop,Portuguese Restaurant,Women's Store,Diner,Discount Store,Distribution Center,Dog Run
2,Downtown Toronto,-79.360636,1,Coffee Shop,Pub,Park,Bakery,Mexican Restaurant,Breakfast Spot,Café,Ice Cream Shop,Electronics Store,Hotel
3,North York,-79.464763,1,Clothing Store,Furniture / Home Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Accessories Store,Vietnamese Restaurant,Convenience Store,Concert Hall
4,Downtown Toronto,-79.389494,1,Coffee Shop,Yoga Studio,Café,Beer Bar,Italian Restaurant,Juice Bar,Boutique,Burger Joint,Burrito Place,Restaurant
7,North York,-79.352188,1,Restaurant,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Gym / Fitness Center,Sporting Goods Shop,Bike Shop,Sandwich Place,Discount Store
8,East York,-79.309937,1,Pizza Place,Gym / Fitness Center,Gastropub,Fast Food Restaurant,Bus Line,Intersection,Café,Bank,Athletics & Sports,Breakfast Spot
9,Downtown Toronto,-79.378937,1,Clothing Store,Coffee Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Cosmetics Shop,Japanese Restaurant,Restaurant,Tea Room,Bookstore
10,North York,-79.445073,1,Park,Pizza Place,Pub,Japanese Restaurant,Eastern European Restaurant,Dumpling Restaurant,Electronics Store,Drugstore,Donut Shop,Department Store
13,North York,-79.340923,1,Restaurant,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Gym / Fitness Center,Sporting Goods Shop,Bike Shop,Sandwich Place,Discount Store
14,East York,-79.318389,1,Skating Rink,Park,Video Store,Pharmacy,Beer Store,Athletics & Sports,Cosmetics Shop,Diner,Curling Ice,Women's Store


In [47]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,-79.329656,2,Park,Fireworks Store,Food & Drink Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
21,York,-79.453512,2,Park,Market,Women's Store,Gourmet Shop,Event Space,Empanada Restaurant,Electronics Store,Greek Restaurant,Eastern European Restaurant,Dumpling Restaurant
35,East York,-79.338106,2,Park,Convenience Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
61,Central Toronto,-79.388790,2,Park,Swim School,Bus Line,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dim Sum Restaurant
66,North York,-79.400049,2,Park,Bank,Bar,Convenience Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
85,Scarborough,-79.284577,2,Park,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store
91,Downtown Toronto,-79.377529,2,Park,Playground,Trail,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Drugstore,Department Store
98,Etobicoke,-79.506944,2,Park,River,Pool,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


In [48]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,-79.532242,3,Baseball Field,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant
101,Etobicoke,-79.498509,3,Baseball Field,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


In [49]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,-79.160497,4,Bar,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


In [50]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,-79.239476,5,Playground,Convenience Store,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
64,York,-79.518188,5,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [51]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,-79.565963,6,Empanada Restaurant,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore


# Conclusion : the cluster 01 there are a lot of interesting points as :parks, restaurants, coffe shops, grocery shops, pharmacy etc... i would be there.